In [51]:
import numpy as np
import matplotlib as plt
from check_link import simple_get
from requests import get
import re
from astropy.coordinates import SkyCoord
import astropy.units as u
from bs4 import BeautifulSoup, Tag

import astropy.units as u
from astropy.coordinates import Distance

In [134]:
class NewGCN:
    __List_of_Tel__ = ["LIGO", "Fermi", "INTEGRAL", "Swift", "MASTER", "BOOTES-4", 
                       "Mondy", "NOT", "AstroSat", "VIRT", "iTelescope",
                       "Insight-HXMT", "Gemini", "Konus", "Zwicky Transient Facility",
                       "ZTF", "Chandra", "AGILE","IceCube", "CALET", "GTC", "KAIT", 
                       "Dabancheng", "GROWTH", "DDOTI", "HAWC","MAXI", "ANTARES"]
    __List_of_Dtr__ = {"Fermi": ["GBM", "LAT"],
                       "Swift": ["BAT", "XRT", "UVOT"],
                       "INTEGRAL": ["IBIS", "SPI"],
                       "LIGO": ["Virgo"],
                       "MASTER": ["MASTER", "NET"],
                       "BOOTES-4": ["MET"],
                       "Mondy": ["Mondy"],
                       "NOT": ["NOT"],
                       "AstroSat": ["CZTI"],
                       "VIRT": ["VIRT"],
                       "iTelescope":["iTelescope"],
                       "Insight-HXMT":["HE"],
                       "Gemini":["Gemini"],
                       "Konus": ["Wind"],
                       "ZTF": ["ZTF"],
                       "Chandra": ["Chandra"],
                       "AGILE": ["GRID", "MCAL"],
                       "IceCube": ["IceCube"],
                       "CALET": ["CGBM"],
                       "GTC": ["GTC"],
                       "KAIT": ["KAIT"],
                       "HMT": ["HMT"],
                       "GROWTH": ["GROWTH"],
                       "DDOTI": ["OAN"],
                       "HAWC": ["HAWC"],
                       "MAXI": ["GSC"],
                       "ANTARES": ["ANTARES"],
                      }
    __Converter__ = {"GBM": [3, 0],
                     "LAT": [4, 0],
                    "IBIS": [3, 0],
                    "SPI": [3, 0],
                    "Virgo" : [-1, 1],
                    "BAT": [3,0],
                    "MASTER":[1,0], 
                    "XRT" :[2,0],
                    "UVOT": [1,0],
                    "MET": [1,0],
                    "Mondy": [1,0],
                    "NOT": [1,0],
                    "CZTI": [2,0],
                    "NET": [1,0],
                    "VIRT": [1,0],
                    "iTelescope": [1,0],
                    "HE": [3, 0],
                    "Gemini": [1,0],
                    "Wind": [2,0],
                    "ZTF": [1,0],
                    "Chandra": [2,0],
                    "GRID":[4, 0],
                    "MCAL": [3,0],
                    "IceCube":[-1, 2],
                    "CALET":[4,0],
                    "CGBM":[4,0],
                    "GTC": [1,0],
                     "KAIT":[1,0],
                     "HMT":[1,0],
                     "GROWTH": [1,0],
                     "OAN": [1,0],
                     "HAWC": [5, 0],
                     "GSC": [2,0],
                     "ANTARES": [-1, 2],
                    }
                     
    __List_of_Month__ = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    __List_of_MW__ = ["radio", "optical", "X-ray", "&gamma;-ray", "HE", "VHE"]
    __List_of_MM__ = ["EM", "GW", "&nu;"]
    Trigger = ''
    Loc = [['',''],['','']]
    Redshift = ['',10]
    scrap = False
    obs = True
    _tr_ = '\t\t\t\t\t\t'
    _td_ = '\t\t\t\t\t\t\t'

    def __init__(self, gcnNum, testmode=False):
        self.link = "https://gcn.gsfc.nasa.gov/gcn3/{}.gcn3".format(gcnNum)
        gcn = simple_get(self.link)
        try:
            gcn = gcn.decode('utf8').strip()
            self.gcn = gcn.split("\n")
        except:
            bs = BeautifulSoup(gcn, 'html.parser')
            gcn = bs.prettify()
            self.gcn = gcn.split("\n")
            
        if testmode:
            raw_html = simple_get(self.link)
            bs = BeautifulSoup(raw_html, 'html.parser')
            print(bs.prettify())
            return
        try:
            tempE = re.findall("(GRB|G|S|IceCube)(-|\s)?([0-9]+)([a-zA-Z]+)[:| -]?", self.gcn[2][9:])[0]
            self._report = True
        except:
            self.__errorMessage__(gcnNum)
            self._report = False
            return
            
        if tempE[0] == "GRB":
            self.evtlink = "https://gcn.gsfc.nasa.gov/other/{}{}.gcn3".format(tempE[2],tempE[3])
        elif tempE[0] == "IceCube":
            self.evtlink = "https://gcn.gsfc.nasa.gov/other/icecube_{}{}.gcn3".format(tempE[2],tempE[3])
        else:
            self.evtlink = "https://gcn.gsfc.nasa.gov/other/{}{}{}.gcn3".format(tempE[0],tempE[2],tempE[3])
            
        self.Event = "{}{}{}{}".format(*tempE)
        self.dates = "{}".format(tempE[2])
        find = False
        self._dtr = []
        self._tel = []
        
        for tel in self.__List_of_Tel__:
            if self.gcn[2].find(tel)>=0:
                if tel == 'Zwicky Transient Facility':
                    self._tel.append('ZTF')
                elif tel == 'Dabancheng':
                    self._tel.append('HMT')
                else:
                    self._tel.append(tel)
        
        if np.size(self._tel) > 1:
            if "LIGO" in self._tel:
                self._tel.remove("LIGO")
            if np.size(self._tel) > 1:
                if "IceCube" in self._tel:
                    self._tel.remove("IceCube")
                if np.size(self._tel) > 1:
                    if "Swift" in self._tel:
                        self._tel.remove("Swift")
        
        if np.size(self._tel) == 0:
            self.__errorMessage__(gcnNum)
            self._report = False
            return
        else:
            self._tel = self._tel[0]

                    
        if np.size(self.__List_of_Dtr__[self._tel]) == 1:
            self._dtr = self.__List_of_Dtr__[self._tel]
        else:
            for dtr in self.__List_of_Dtr__[self._tel]:
                if dtr in self.gcn[2]:
                    self._dtr.append(dtr)
                    find = True
            if not(find):
                for dtr in self.__List_of_Dtr__[self._tel]:
                    for line in self.gcn[2:]:
                        if (line.find(dtr)>=0) and (dtr not in self._dtr):
                            self._dtr.append(dtr)
                            find=True
            if not(find):
                self._dtr = self.__List_of_Dtr__[self._tel]

        if np.size(self._dtr) == 1:
            self._dtr = self._dtr[0]
        elif self._tel == '':
            self._report = False
            return
        
        self._multidtr = np.zeros([2,np.size(self._dtr)], dtype = 'bool')
        self._multidtr[1] = True
        
        if (np.size(self._dtr)>1):
            if len(re.findall(" (No|no|Upper|upper|limit|Retraction|inactive|SAA) ", self.gcn[2]))>0:
                self._multidtr[1] = True
                self._multidtr[1] = False
            else:
                for line in self.gcn[5:]:
                    for i in range(np.size(self._dtr)):
                        if line.find(self._dtr[i])>=0:
                            self._multidtr[0][i] = 1
                    if (sum(self._multidtr[0])>0) and (len(re.findall("(no.candidates|no.candidate|upper|not.detect|Retraction)", line))>0):
                        self._multidtr[1][sum(self._multidtr[0])-1] = 0 
            self.obs = sum(self._multidtr[1])
        else:
            if np.size(re.findall("\s(No|no|Upper|upper|limit|Retraction|inactive|SAA)\s?", self.gcn[2]))>0:
                self.obs = False 
            else:
                contents = str("")
                for line in self.gcn[5:]:
                    contents= contents+" "+str(line)
                if np.size(re.findall("\s(no.candidates|no.candidate|no.credible|Upper|upper|not.detect)\s?", contents))>0:
                    self.obs = False                 
        
    def __errorMessage__(self, gcnNum):
        print("This GCN ({}) may not be related to GRB, GW, or neutrinos. Or, the telescope is not in our list".format(gcnNum))
        print("Please check the GCN.")
        print("Title: {}".format(self.gcn[2][8:]))
        print("Link: {}".format(self.link))
        return
                
    def __scrapping__(self):
        
        takeY = self.dates[:2]
        takeM = self.dates[2:4]
        takeD = self.dates[4:]

        if not(self.obs):
            self._errC = 10000
            self.Trigger = "20{}-{}-{}".format(takeY, takeM, takeD)
            return 0;

        T0Type = 1
        LocType = 0
        ZType = 0
        tempErr = [-1, 'degrees']
        breakP = 0
        

        for line in self.gcn[5:]:
            if T0Type ==1:
                tempT = re.findall("[At|at]?\s?([0-9]+):([0-9]+):([0-9.]+)", line)
                if len(tempT)>0:
                    takeT = tempT[0]
                    T0Type = 2
                    break
    
        for line in self.gcn[5:]:
            try:
                tempRa = re.findall("[RA|RA(J2000)|RA  (J2000)]\s?\s?\s?[:|=]\s?\s?\s?([0-9]+)[:|h|\s]\s?([0-9]+)[:|m|\s]\s?([0-9.]+)", line[line.find("RA"):])[0]
                LocType = 1
            except:
                try:
                    tempRa = re.findall("[RA|RA(J2000)|RA  (J2000)]\s?\s?\s?[:|=]\s?\s?([0-9.]+)", line[line.find("RA"):])[0]                   
                    LocType = 2
                except:
                    pass

            try:
                tempDec = re.findall("[Dec|DECL.|Dec(J2000)|Dec (J2000)]\s?[:|=]\s?([\-|\+]?)([0-9]+)[:|d|\s]\s?([0-9]+)[:|m|'|\s|’]\s?([0-9.]+)", line[line.find("Dec"):])[0]
            except:
                try:
                    tempDec = re.findall("[Dec|DECL.|Dec(J2000)|Dec (J2000)]\s?[:|=]\s?([\-|\+]?)([0-9.]+)", line[line.find("Dec"):])[0]
                except:
                    pass
            
            try:
                tempLoc = re.findall("[RA, Dec]\s?[:|=]\s?([0-9.]+)\,\s?([\-|\+]?)\s?([0-9.]+)", line[line.find("RA"):])[0]
                tempRa = tempLoc[0]
                tempDec = tempLoc[1:]
                LocType = 2
            except:
                pass
            
            try:
                tempErr = re.findall("[uncertainty|error radius] of [about ]?([0-9.]+) (deg|degrees|arcmin|arcseconds|arcsec|arc sec)", line)[0]
                break
            except:
                try:
                    bitempErr = re.findall("\+([0-9.]+)\/\-([0-9.]+) (deg|degrees|arcmin|arcseconds|arc sec)", line)[0]
                    tempErr = [max(float(tempErr[0]), float(bitempErr[0]), float(bitempErr[1])), bitempErr[2]]
                    breakP += 1
                    if breakP == 2: 
                        break
                except:
                    pass

        for line in self.gcn[5:]:
            try:
                tempZ = re.findall("z = ([0-9.]+) \+\/\- ([0-9.]+)", line)[0]
                ZType = 1
            except:
                try:
                    tempZ = re.findall("estimate is ([0-9.]+) \+\/\- ([0-9.]+) (Mpc|kpc)", line)[0]
                    ZType = 2
                except:
                    pass
            
        if T0Type == 1:
            self.Trigger = "20{}-{}-{}".format(takeY, takeM, takeD)
        if T0Type == 2:
            self.Trigger = "20{}-{}-{} {}:{}:{}".format(takeY, takeM, takeD, *takeT)
        
        if LocType == 1:
            ra = "{}:{}:{}".format(*tempRa)
            dec = "{}{}:{}:{}".format(*tempDec)
            c = SkyCoord(ra, dec, unit=(u.hourangle,  u.deg))
            self.Loc = [["{:.3f}".format(c.ra.deg), "{:.3f}".format(c.dec.deg)], [ra, dec]]
        elif LocType == 2:
            ra = float("{}".format(tempRa))
            dec = float("{}{}".format(*tempDec))
            c = SkyCoord(ra*u.deg, dec*u.deg)
            self.Loc = [[ra, dec], ["{:.0f}:{:.0f}:{:.2f}".format(*c.ra.hms), "{:.0f}:{:.0f}:{:.2f}".format(c.dec.hms[0], abs(c.dec.hms[1]), abs(c.dec.hms[2]))]]
        
        if float(tempErr[0])>0:
            self._errC = float("{}".format(tempErr[0]))
            if tempErr[1] in ['degrees', 'deg']:
                self._errC*=60
            elif tempErr[1] in ['arcseconds', 'arcsec', 'arc sec']:
                self._errC/=60  
        else:
            self._errC = 10000
            
        if ZType == 1:
            self.Redshift = ['{:.4f}'.format(float(tempZ[0])), float(tempZ[1])]
        elif ZType == 2:
            if tempZ[2] == 'Mpc':
                unit = u.Mpc
            elif tempZ[2] == 'kpc':
                unit = u.kpc
            d0 = Distance(float(tempZ[0]), unit=unit)
            z0 = d0.compute_z()
            derr1 = Distance(float(tempZ[0])+float(tempZ[1]), unit=unit)
            zerr1 = derr1.compute_z()
            derr2 = Distance(float(tempZ[0])-float(tempZ[1]), unit=unit)
            zerr2 = derr2.compute_z()
            print(z0, zerr1,zerr2, tempZ, float(tempZ[0])+float(tempZ[1]), float(tempZ[0])-float(tempZ[1]))
            zerr = max(abs(zerr1-z0), abs(z0-zerr2))
            self.Redshift = ['{:.4f}'.format(z0), zerr]  
                
    def __genhtml__(self, link, tooltip, content, err = -1):
        html = ''
        if link != '':
            html+='<a href="{}" style="color:#808080" target="gcn">'.format(link)
        if err >=0:
            html+='<div data-toggle="tooltip" title="{}" data-placement="bottom" err={}>{}</div>'.format(tooltip, err, content)
        else:
            html+='<div data-toggle="tooltip" title="{}" data-placement="bottom">{}</div>'.format(tooltip, content)
        if link !='':
            html+='</a>'
        return html
                    
    def __genTable__(self):
        tab = '<!--newevt-->\n'
        tab +=self._tr_+'<tr class="row100 body" id="{}">\n'.format(self.Event)
        
        # Event name
        if "IceCube" in self.Event:
            tab +=self._td_+'<td class="cell100 evt"><a href="{}" style="color:#808080" target="gcn">IC{}</a></td>\n'.format(self.evtlink, self.Event[8:])
        else:
            tab +=self._td_+'<td class="cell100 evt"><a href="{}" style="color:#808080" target="gcn">{}</a></td>\n'.format(self.evtlink, self.Event)
        
        # Trigger time
        if (np.size(self._dtr) == 1) and (self._tel != self._dtr):
            reportName = "{}/{}".format(self._tel, self._dtr)
        elif np.size(self._dtr) > 1:
            tempName = ''
            for i in range(self.obs):
                tempName += "{}/{} or ".format(self._tel, self._dtr[i])
            reportName = tempName[:-4]
        else:
            reportName = "{}".format(self._tel)
        
        tab +=self._td_+'<td class="cell100 t0">'+self.__genhtml__(self.link, reportName, self.Trigger)+'</td>\n'
        
        # Localization
        if self.Loc[0][0] == '':
            tab +=self._td_+'<td class="cell100 ra">'+self.__genhtml__('', self.Loc[1][0], self.Loc[0][0], err=self._errC)+'</td>\n'
            tab +=self._td_+'<td class="cell100 dec">'+self.__genhtml__('', self.Loc[1][1], self.Loc[0][1])+'</td>\n'
        else:
            tab +=self._td_+'<td class="cell100 ra">'+self.__genhtml__('', self.Loc[1][0], str(self.Loc[0][0])+' &deg;', err=self._errC)+'</td>\n'
            tab +=self._td_+'<td class="cell100 dec">'+self.__genhtml__('', self.Loc[1][1], str(self.Loc[0][1])+' &deg;')+'</td>\n'
        
        if self._errC == 10000:
            err = ""
        elif self._errC > 6:
            err = "{:.2f} &deg;".format(self._errC/60.)
        elif self._errC < 0.6:
            err = '{:.2f} "'.format(self._errC*60.)
        else:
            err = "{:.2f} '".format(self._errC)
            
        tab +=self._td_+'<td class="cell100 err">'+self.__genhtml__(self.link, reportName, err)+'</td>\n'
        
        # Redshift
        if self.Redshift[1] == 10:
            tab +=self._td_+'<td class="cell100 z">'+self.__genhtml__('', '', self.Redshift[0], err=self.Redshift[1])+'</td>\n'
        else:
            tab +=self._td_+'<td class="cell100 z">'+self.__genhtml__(self.link, '+/-{:.4f}'.format(self.Redshift[1]), self.Redshift[0], err=self.Redshift[1])+'</td>\n'

        # Telescope/detector    
        tab +=self._td_+'<td class="cell100 tel">'
        
        if self.obs:
            if (np.size(self._dtr)>1):
                mw = []
                mm = []
                for i in range(np.size(self._dtr)):
                    if self._multidtr[1][i]: 
                        tab +='<a href="{}" target="gcn" ><div class="box_tel_on">{}/{}</div></a>'.format(self.link, self._tel, self.__List_of_Dtr__[self._tel][i])
                        mw.append(self.__Converter__[self.__List_of_Dtr__[self._tel][i]][0])
                        mm.append(self.__Converter__[self.__List_of_Dtr__[self._tel][i]][1])
                    else:
                        tab +='<a href="{}" target="gcn" ><div class="box_tel_off">{}/{}</div></a>'.format(self.link, self._tel, self.__List_of_Dtr__[self._tel][i])
            else:
                if self._tel == self._dtr:
                    tab +='<a href="{}" target="gcn" ><div class="box_tel_on">{}</div></a>'.format(self.link, self._tel)
                else:
                    tab +='<a href="{}" target="gcn"><div class="box_tel_on">{}/{}</div></a>'.format(self.link, self._tel, self._dtr)
                mw = [self.__Converter__[self._dtr][0]]
                mm = [self.__Converter__[self._dtr][1]]
        else:
            if self._tel == self._dtr:
                tab +='<a href="{}" target="gcn"><div class="box_tel_off">{}</div></a>'.format(self.link, self._tel)
            else:
                tab +='<a href="{}" target="gcn"><div class="box_tel_off">{}/{}</div></a>'.format(self.link, self._tel, self._dtr)
            mw = [-1]
            mm = [-1]
            
        
        # Wavelength
        tab += '</td>\n'+self._td_+'<td class="cell100 mw">'

        for i in range(6):
            if i in mw:
                tab +='<div class="box_{}">{}</div>'.format(i, self.__List_of_MW__[i])
            else:
                tab +='<div class="box_off">{}</div>'.format(self.__List_of_MW__[i])

        # Messenger
        tab += '</td>\n'+self._td_+'<td class="cell100 mm">'
        
        for i in range(3):
            if i in mm:
                if i == 1:
                    ligolink = "https://gracedb.ligo.org/superevents/{}/view/".format(self.Event)
                    tab+='<a href="{}" target="_blank"><div class="box_on">{}</div></a>'.format(ligolink, self.__List_of_MM__[i])
                else:
                    tab +='<div class="box_on">{}</div>'.format(self.__List_of_MM__[i])
            else:
                tab +='<div class="box_off">{}</div>'.format(self.__List_of_MM__[i])
            
        tab +='</td>\n'+self._tr_+'</tr>\n'
        return tab
    
    def __updateTable__(self, newhtml, verbose=False, testmode=False):
        
        templist = newhtml.findAll("tr",{"id":"{}".format(self.Event)})[0]
        
        changes = []
        
        if testmode:
            return templist
        
        if (np.size(self._dtr) == 1) and (self._tel != self._dtr):
            reportName = "{}/{}".format(self._tel, self._dtr)
        elif np.size(self._dtr) > 1:
            tempName = ''
            for i in range(self.obs):
                tempName += "{}/{} or ".format(self._tel, self._dtr[i])
            reportName = tempName[:-4]
        else:
            reportName = "{}".format(self._tel)
        
        # Trigger time
        if (self._tel == "LIGO") and self.obs:
            changes.append(self._td_+'<td class="cell100 t0">'+self.__genhtml__(self.link, reportName, self.Trigger)+'</td>\n')
        else:
            refT0_text = templist.find_all("td", {"class":"cell100 t0"})[0].text
            refT0 = re.findall('[0-9]+\-[0-9]+\-[0-9]+\s([0-9]+)?:?([0-9]+)?:?([0-9.]+)?', refT0_text)
            newT0 = re.findall('[0-9]+\-[0-9]+\-[0-9]+\s([0-9]+)?:?([0-9]+)?:?([0-9.]+)?', self.Trigger)

            if (np.size(refT0) == 0) or (np.size(newT0) == 0):
                if len(self.Trigger) > 10:
                    changes.append(self._td_+'<td class="cell100 t0">'+self.__genhtml__(self.link, reportName, self.Trigger)+'</td>\n')
                else:
                    changes.append(self._td_+str(templist.find_all("td", {"class":"cell100 t0"})[0])+'\n')
            else:
                refT0 = float('{}{}{}'.format(*refT0[0]))
                newT0 = float('{}{}{}'.format(*newT0[0]))
                
                if (abs(refT0 - newT0) > 100) or (refT0<=newT0):
                    changes.append(self._td_+str(templist.find_all("td", {"class":"cell100 t0"})[0])+'\n')
                else:
                    changes.append(self._td_+'<td class="cell100 t0">'+self.__genhtml__(self.link, reportName, self.Trigger)+'</td>\n')
        
        # Localization
        refra = templist.find_all("td", {"class":"cell100 ra"})[0].text[:-1]
        refdec = templist.find_all("td", {"class":"cell100 dec"})[0].text[:-1]
        refra_hms = templist.find_all("td", {"class":"cell100 ra"})[0].find("div").attrs['title']
        refdec_hms = templist.find_all("td", {"class":"cell100 dec"})[0].find("div").attrs['title']
        refErrC = templist.find_all("td", {"class":"cell100 ra"})[0].find("div").attrs['err']
        
        refLoc = [[refra, refdec], [refra_hms, refdec_hms]]
        LocF = False

        if (self.obs) and (float(self._errC) <= float(refErrC)):            
            updatedLoc = self.Loc
            updatederrC = self._errC
            LocF = True
            if verbose:
                print("Localization is updated.")
        else:
            updatedLoc = refLoc
            updatederrC = float(refErrC)
            
        if updatederrC == 10000:
            updatederr = ""
        elif updatederrC > 6:
            updatederr = "{:.2f} &deg;".format(updatederrC/60.)
        elif updatederrC < 0.6:
            updatederr = '{:.2f} "'.format(updatederrC*60.)
        else:
            updatederr = "{:.2f} '".format(updatederrC)
            
        if updatedLoc[0][0] == '':
            changes.append(self._td_+'<td class="cell100 ra">'+self.__genhtml__('', updatedLoc[1][0], updatedLoc[0][0], err=updatederrC)+'</td>\n')
            changes.append(self._td_+'<td class="cell100 dec">'+self.__genhtml__('', updatedLoc[1][1], updatedLoc[0][1])+'</td>\n')
            
        else:
            changes.append(self._td_+'<td class="cell100 ra">'+self.__genhtml__('', updatedLoc[1][0], str(updatedLoc[0][0])+' &deg;', err=updatederrC)+'</td>\n')
            changes.append(self._td_+'<td class="cell100 dec">'+self.__genhtml__('', updatedLoc[1][1], str(updatedLoc[0][1])+' &deg;')+'</td>\n')
            
        if LocF == True:
            changes.append(self._td_+'<td class="cell100 err">'+self.__genhtml__(self.link, reportName, updatederr)+'</td>\n')
        else:
            changes.append(self._td_+str(templist.find_all("td", {"class":"cell100 err"})[0])+'\n')
            
        # Redshift
        refZ = templist.find_all("td", {"class":"cell100 z"})[0].text
        refErrZ = float(templist.find_all("td", {"class":"cell100 z"})[0].find_all("div")[0].attrs['err'])
        refErrText = templist.find_all("td", {"class":"cell100 z"})[0].find("div").attrs['title']
        
        if (self.obs) and (refErrZ > self.Redshift[1]):
            changes.append(self._td_+'<td class="cell100 z">'+self.__genhtml__(self.link, '+/-{:.4f}'.format(self.Redshift[1]), self.Redshift[0], err=self.Redshift[1])+'</td>\n')
        else:
            changes.append(self._td_+str(templist.find_all("td", {"class":"cell100 z"})[0])+'\n')
        
        # Telescope/detector
        if self._tel == self._dtr:
            fullname_dtr = "{}".format(self._tel)
            reftel = self.__addDtrReport__(newhtml, fullname_dtr, self.obs)
        else:
            if np.size(self._dtr) == 1:
                fullname_dtr = "{}/{}".format(self._tel, self._dtr)
                reftel = self.__addDtrReport__(newhtml, fullname_dtr, self.obs)
            else:
                for i in range(np.size(self._dtr)):
                    fullname_dtr = "{}/{}".format(self._tel, self._dtr[i])
                    obs = self._multidtr[1][i]
                    reftel = self.__addDtrReport__(newhtml, fullname_dtr, obs)                    

        changes.append(self._td_+'{}\n'.format(reftel))
        
        # Wavelength
        if self.obs:
            if (np.size(self._dtr)>1):
                mw = []
                mm = []
                for i in range(np.size(self._dtr)):
                    if self._multidtr[1][i]: 
                        mw.append(self.__Converter__[self.__List_of_Dtr__[self._tel][i]][0])
                        mm.append(self.__Converter__[self.__List_of_Dtr__[self._tel][i]][1])
            else:
                mw = [self.__Converter__[self._dtr][0]]
                mm = [self.__Converter__[self._dtr][1]]
        else:
            mw = [-1]
            mm = [-1]

        refmw = templist.find_all("td", {"class":"cell100 mw"})[0]

        for pre_mw, i in zip(refmw.find_all("div"), range(len(refmw.find_all("div")))):
            if pre_mw.attrs['class'][0] != 'box_off':
                mw.append(i)
        
        modifiedmw = self._td_+'<td class="cell100 mw">'
        for i in range(6):
            if i in mw:
                modifiedmw +='<div class="box_{}">{}</div>'.format(i, self.__List_of_MW__[i])
            else:
                modifiedmw +='<div class="box_off">{}</div>'.format(self.__List_of_MW__[i])
        modifiedmw+='</td>\n'
        changes.append(str(modifiedmw))

        # Messenger
        refmm = templist.find_all("td", {"class":"cell100 mm"})[0]
        
        if self._tel == 'LIGO' and self.obs == False:
            refmm.find_all("div")[1].attrs['class'] = 'box_off'
        elif self._tel == 'IceCube' and self.obs == False:
            refmm.find_all("div")[2].attrs['class'] = 'box_off'
        changes.append(self._td_+'{}\n'.format(str(refmm)))
        
        return changes

    def __addDtrReport__(self, newhtml, fullname_dtr, obs):
        templist = newhtml.findAll("tr",{"id":"{}".format(self.Event)})[0]
        reftel = templist.find_all("td", {"class":"cell100 tel"})[0]
        reported = [det.text for det in reftel.find_all("div")]
        if fullname_dtr in reported:
            for link in reftel.find_all("a"):
                if link.text == fullname_dtr:
                    link.attrs['href'] = self.link
                    if obs:
                        link.find("div").attrs['class'] = "box_tel_on"

        else:
            if obs:
                new_tel = newhtml.new_tag("div", **{'class':'box_tel_on'})
                new_tel_link = newhtml.new_tag("a", **{'href':'{}'.format(self.link), 'target':"gcn"})
                new_tel.string = fullname_dtr
                new_tel_link.insert(1, new_tel)
            else:
                new_tel = newhtml.new_tag("div", **{'class':'box_tel_off'})
                new_tel_link = newhtml.new_tag("a", **{'href':'{}'.format(self.link), 'target':"gcn"})
                new_tel.string = fullname_dtr
                new_tel_link.insert(1, new_tel)

            reftel.append(new_tel_link)
            
        dtr_on = []
        dtr_off = []
        for dtr in reftel.find_all("a"):
            if dtr.find("div").attrs['class'][0]=="box_tel_on":
                dtr_on.append(dtr)
            else:
                dtr_off.append(dtr)

        reftel = '<td class="cell100 tel">'
        for dtr in dtr_on:
            reftel+=str(dtr)
        for dtr in dtr_off:
            reftel+=str(dtr)
        reftel +='</td>'

        return reftel
        
    def uploadGCN(self, verbosity=False, testmode=False):
        
        if self._report:
            self.__scrapping__()
        else:
            return
        
        if verbosity:
            if verbosity == 2:
                raw_html = simple_get(self.link)
                bs = BeautifulSoup(raw_html, 'html.parser')
                print(bs.prettify())
            print("Telescope: {}".format(self._tel))
            print("Detector: {}".format(self._dtr)) 
            if self.obs: 
                print("Messenger is detected.")
                print("Trigger: {}".format(self.Trigger))
            if self.obs and self._errC != 10000:
                print("Localization: RA = {} / Dec = {} / Error = {:.3f} arcmin".format(self.Loc[0][0], self.Loc[0][1], self._errC))
            
        raw_html = open('./templates/table/index.html', 'r')
        newhtml = BeautifulSoup(raw_html, 'html.parser')

        Eventlist = []
        templist = newhtml.findAll("tr", {"class":"row100 body"})
        for evt in templist:
            Eventlist.append(evt.attrs['id'])
        raw_html.close()

        if self.Event not in Eventlist:
            newtab = self.__genTable__()
            raw_html = open('./templates/table/index.html', 'r')
            new_html = ''
            for line in raw_html.readlines():
                if re.findall("<!--newevt-->", line):
                    new_html+=newtab
                else:
                    new_html+=line

            with open('./templates/table/index.html' ,'w') as raw_html:
                raw_html.write(new_html)
        else:
            changes = self.__updateTable__(newhtml, verbose=verbosity, testmode=testmode)
            if testmode:
                return changes
            raw_html = open('./templates/table/index.html', 'r')
            new_html = ''
            i=0
            evtflag=False
            for line in raw_html.readlines():
                if re.findall(self.Event+'">', line):
                    evtflag = True
                    i=0
                if evtflag and i>=2:
                    new_html+=changes[i-2]
                    if i>=8:
                        evtflag=False
                    i+=1
                else:
                    i+=1
                    new_html+=line
            with open('./templates/table/index.html' ,'w') as raw_html:
                raw_html.write(new_html)

In [135]:
newevt = NewGCN(25497)
newevt.uploadGCN()

newevt = NewGCN(25503)
newevt.uploadGCN()

newevt = NewGCN(25324)
newevt.uploadGCN()

for i in range(25551, 25566):
    print(i)
    newevt = NewGCN(i)
    newevt.uploadGCN()

0.40561664631713157 0.485006292168464 0.32203881721544725 ('2276', '538', 'Mpc') 2814.0 1738.0
0.3012396432995967 0.3687626091806021 0.22999159192610388 ('1609', '426', 'Mpc') 2035.0 1183.0
0.05968622492667808 0.07122097096861495 0.04797306091778892 ('276', '56', 'Mpc') 332.0 220.0
25551
25552
25553
This GCN (25553) may not be related to GRB, GW, or neutrinos. Or, the telescope is not in our list
Please check the GCN.
Title:  Fermi  trigger No 588801358 Global MASTER-Net observations report 
Link: https://gcn.gsfc.nasa.gov/gcn3/25553.gcn3
25554
25555
25556
25557
25558
25559
25560
25561
25562
25563
25564
25565


In [136]:
newevt = NewGCN(26269)
newevt.uploadGCN()
for i in range(26276, 26322):
    print(i)
    newevt = NewGCN(i)
    newevt.uploadGCN()

26276
26277
26278
26279
26280
26281
26282
26283
26284
26285
26286
26287
26288
26289
26290
26291
This GCN (26291) may not be related to GRB, GW, or neutrinos. Or, the telescope is not in our list
Please check the GCN.
Title:  Baksan Neutrino Observatory Alert 191124.13: Global MASTER-Net observations report 
Link: https://gcn.gsfc.nasa.gov/gcn3/26291.gcn3
26292
26293
26294
This GCN (26294) may not be related to GRB, GW, or neutrinos. Or, the telescope is not in our list
Please check the GCN.
Title:  Fermi  trigger No 596387570: Global MASTER-Net observations report 
Link: https://gcn.gsfc.nasa.gov/gcn3/26294.gcn3
26295
26296
26297
This GCN (26297) may not be related to GRB, GW, or neutrinos. Or, the telescope is not in our list
Please check the GCN.
Title:  Baksan Neutrino Observatory Alert 191127.25: Global MASTER-Net observations report 
Link: https://gcn.gsfc.nasa.gov/gcn3/26297.gcn3
26298
26299
26300
This GCN (26300) may not be related to GRB, GW, or neutrinos. Or, the telescope is 

In [132]:
newevt = NewGCN(25501)
newevt.uploadGCN()
